In [0]:
sc

<SparkContext master=spark://10.139.64.6:7077 appName=Databricks Shell>

In [0]:
spark

In [0]:
taxi_df = spark.read.csv("/Volumes/azure_databricks/default/greentaxi/2021_Green_Taxi_Trip_Data.csv",header=True,inferSchema=True)

In [0]:
taxi_df.repartition(10) \
    .write \
    .mode("append") \
    .option("header", "true") \
    .parquet("/Volumes/azure_databricks/default/greentaxi/")

In [0]:
taxi_df.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('lpep_pickup_datetime', StringType(), True), StructField('lpep_dropoff_datetime', StringType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', IntegerType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', StringType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', IntegerType(), True), StructField('trip_type', IntegerType(), True), StructField('congestion_surcharge', Doubl

In [0]:
# taxiReadStream = spark.readStream.format("parquet").schema(taxi_df.schema).option("header","true").load("/Volumes/azure_databricks/default/readstream/")
# # Read stream transformation
# taxiTransformation = taxiReadStream.groupBy("trip_type").sum("fare_amount")


In [0]:
# taxiwriteStream = taxiReadStream.writeStream.outputMode("append").format("parquet").option("path","/Volumes/azure_databricks/default/writestream/").option("checkpointlocation","/Volumes/azure_databricks/default/checkpoint/").start()

# taxiwriteStream.awaitTermination()

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:718)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:437)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:437)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

In [0]:
from pyspark.sql.functions import *
# Step 1: Create a ReadStream
taxiReadStream = spark.readStream.format("parquet").schema(taxi_df.schema).option("header", True).load("/Volumes/azure_databricks/default/readstream/")

# Step 2: Convert the string to a timestamp
taxiReadStream = taxiReadStream.withColumn(
    "lpep_pickup_datetime", 
    to_timestamp(col("lpep_pickup_datetime"), "yyyy-MM-dd HH:mm:ss")
)

# Step 3: Add a watermark and perform the aggregation (window of aggregate operations)
taxiTransformation = taxiReadStream \
    .withWatermark("lpep_pickup_datetime", "10 minutes") \
    .groupBy(
        window(col("lpep_pickup_datetime"), "10 minutes"),
        col("trip_type")
    ).sum("fare_amount")
    
# Step 4: Write the aggregated data to Parquet files
taxiTransformation.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "/Volumes/azure_databricks/default/writestream/") \
    .option("checkpointLocation", "/Volumes/azure_databricks/default/checkpoint/") \
    .start() \
    .awaitTermination()



com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:718)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:437)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:437)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

In [0]:
from pyspark.sql.functions import to_timestamp, col
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Step 1: Read the streaming data from Parquet files and create a read stream 
taxiReadStream = spark.readStream.format("parquet") \
    .schema(taxi_df.schema) \
    .option("header", True) \
    .load("/Volumes/azure_databricks/default/readstream/")

# Step 2: Convert the string to a timestamp
taxiReadStream = taxiReadStream.withColumn(
    "lpep_pickup_datetime", 
    to_timestamp(col("lpep_pickup_datetime"), "yyyy-MM-dd HH:mm:ss")
)

# Step 3: Add a watermark and perform aggregation everyth transformation has a watermark and window function
taxiReadStream = taxiReadStream.withWatermark("lpep_pickup_datetime", "10 minutes")
# taxiTransformation = taxiReadStream\
#     .withWatermark("lpep_pickup_datetime", "10 minutes")\
#         .groupBy(window(col(""lpep_pickup_datetime"), "10 minutes"),
#                             col("trip_type")).sum("fare_amount"))
taxiReadStream.createOrReplaceTempView("readdata")

# Step 4: Perform the aggregation (window of aggregate operations)
# Corrected the SQL query to use window functions properly
aggregatedData = spark.sql("""
    select lpep_pickup_datetime,trip_type,count(*) from readdata where fare_Amount > 30 group by lpep_pickup_datetime,trip_type
""")

# Step 5: Write the aggregated data to Parquet files
aggregatedData.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "/Volumes/azure_databricks/default/writestream/") \
    .option("checkpointLocation", "/Volumes/azure_databricks/default/checkpoint/") \
    .start() \
    .awaitTermination()

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:103)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2(SequenceExecutionState.scala:103)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$2$adapted(SequenceExecutionState.scala:100)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:100)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:718)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:437)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:437)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

In [0]:
parq_df = "/Volumes/azure_databricks/default/writestream/"
df = spark.read.parquet(parq_df,header=True)
df.show()

+--------------------+---------+--------+
|lpep_pickup_datetime|trip_type|count(1)|
+--------------------+---------+--------+
+--------------------+---------+--------+



read from blod transforrm in db upload in cosmos db

1. Create a dataset

In [0]:
columns = ["Product_Code", "Product_Name", "Sales", "Date_and_Time", "Profit", "Payment_Method"]
data = [
    ("P001", "Laptop", 36, "2023-02-16 14:36:19.196", 92.55, "Debit Card"),
    ("P002", "Smartphone", 58, "2023-06-02 09:15:32.421", 75.20, "Credit Card"),
    ("P003", "Tablet", 4, "2023-05-23 14:36:19.196", 65.95, "Credit Card"),
    ("P004", "Headphones", 79, "2023-03-12 14:36:19.196", 49.73, "Cash"),
    ("P005", "Smartwatch", 22, "2023-07-18 11:42:05.789", 38.60, "PayPal"),
    ("P006", "Printer", 12, "2023-08-30 16:20:45.123", 82.15, "Debit Card"),
    ("P007", "Monitor", 31, "2023-01-25 13:50:22.987", 105.30, "Bank Transfer"),
    ("P008", "Keyboard", 67, "2023-11-03 08:30:15.741", 25.80, "Cash"),
    ("P009", "Mouse", 85, "2023-10-07 12:10:33.852", 18.95, "PayPal"),
    ("P010", "External HDD", 14, "2023-07-09 14:15:50.753", 61.20, "Cash"),
    ("P011", "Webcam", 29, "2023-09-05 13:20:18.951", 33.70, "PayPal"),
    ("P012", "Speaker", 46, "2023-01-09 17:10:43.951", 45.90, "Bank Transfer"),
    ("P013", "Graphics Card", 18, "2023-12-20 15:55:49.369", 272.40, "Credit Card"),
    ("P014", "Router", 27, "2023-05-11 17:40:28.963", 88.75, "Debit Card"),
    ("P015", "Docking Station", 39, "2023-02-28 09:25:17.159", 142.30, "Credit Card"),
    ("P016", "Portable SSD", 72, "2023-08-22 11:30:42.951", 179.85, "Bank Transfer"),
    ("P017", "Wireless Earbuds", 61, "2023-06-30 10:50:25.753", 152.15, "Credit Card"),
    ("P018", "Gaming Console", 52, "2023-11-28 15:45:37.159", 397.80, "Cash"),
    ("P019", "Fitness Tracker", 23, "2023-04-03 08:55:29.753", 112.40, "Debit Card"),
    ("P020", "Portable Charger", 33, "2023-12-12 12:35:51.357", 68.25, "Credit Card"),
    ("P021", "VR Headset", 55, "2023-10-20 09:40:35.753", 539.75, "Cash"),
    ("P022", "Drone", 41, "2023-03-06 11:15:48.357", 827.35, "PayPal"),
    ("P023", "Smart Home Hub", 69, "2023-05-19 16:50:39.951", 184.20, "Debit Card"),
    ("P024", "Projector", 19, "2023-08-14 10:30:27.753", 758.80, "Cash"),
    ("P025", "E-reader", 74, "2023-11-09 13:05:19.159", 154.45, "Credit Card"),
    ("P026", "Digital Camera", 93, "2023-02-23 15:40:56.357", 620.10, "Bank Transfer"),
    ("P027", "Soundbar", 15, "2023-06-17 08:20:43.951", 289.70, "Debit Card"),
    ("P028", "Smart Scale", 26, "2023-09-28 12:55:35.753", 70.85, "PayPal"),
    ("P029", "WiFi Extender", 37, "2023-04-22 17:30:22.159", 85.50, "Cash"),
    ("P030", "Game Controller", 64, "2023-12-05 09:10:18.357", 77.15, "Credit Card")]


In [0]:
df = spark.createDataFrame(data,columns)
df.show()

In [0]:
from pyspark.sql import Row
# define a case class
Sales = Row("Product_Code", "Product_Name", "Sales", "Date_and_Time", "Profit", "Payment_Method")
datasets = df.rdd.map(lambda row : Sales(*row)).toDF()

In [0]:
datasets.show()